In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('wgan')):
    os.mkdir('saved_models_mnist_{}'.format('wgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('wgan'), mode='w')
import os
import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt
import os
import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
                            optimizer=optimizer,
                            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer,
                              metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test,y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):


            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    # idx = np.random.randint(0, X_train.shape[0], batch_size)
                    # imgs = X_train[idx]

                    # Sample noise as generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                    # Generate a batch of new images
                    gen_imgs = self.generator.predict(noise)

                    # Train the critic
                    d_loss_real = self.critic.train_on_batch(imgs, valid)
                    d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                    # Clip critic weights
                    for l in self.critic.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                        l.set_weights(weights)

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f] [G loss: %f]" % (epoch,global_step, 1 - d_loss[0], 1 - g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=20, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:5[D loss: 0.999922] [G loss: 1.000159]
epoch:0 step:10[D loss: 0.999928] [G loss: 1.000164]
epoch:0 step:15[D loss: 0.999927] [G loss: 1.000168]
epoch:0 step:20[D loss: 0.999927] [G loss: 1.000163]
epoch:0 step:25[D loss: 0.999931] [G loss: 1.000167]
epoch:0 step:30[D loss: 0.999928] [G loss: 1.000167]
epoch:0 step:35[D loss: 0.999930] [G loss: 1.000167]
epoch:0 step:40[D loss: 0.999925] [G loss: 1.000166]
epoch:0 step:45[D loss: 0.999925] [G loss: 1.000168]
epoch:0 step:50[D loss: 0.999928] [G loss: 1.000167]
epoch:0 step:55[D loss: 0.999927] [G loss: 1.000166]
epoch:0 step:60[D loss: 0.999932] [G loss: 1.000170]
epoch:0 step:65[D loss: 0.999930] [G loss: 1.000163]
epoch:0 step:70[D loss: 0.999932] [G loss: 1.000151]
epoch:0 step:75[D loss: 0.999939] [G loss: 1.000151]
epoch:0 step:80[D loss: 0.999936] [G loss: 1.000144]
epoch:0 step:85[D loss: 0.999937] [G loss: 1.000131]
epoch:0 step:90[D loss: 0.999938] [G loss: 1.000126]
epoch:0 step:95[D loss: 0.999943] [G loss: 1.00

epoch:0 step:705[D loss: 0.999975] [G loss: 1.000065]
epoch:0 step:710[D loss: 0.999972] [G loss: 1.000070]
epoch:0 step:715[D loss: 0.999970] [G loss: 1.000069]
epoch:0 step:720[D loss: 0.999976] [G loss: 1.000073]
epoch:0 step:725[D loss: 0.999973] [G loss: 1.000072]
epoch:0 step:730[D loss: 0.999972] [G loss: 1.000068]
epoch:0 step:735[D loss: 0.999974] [G loss: 1.000072]
epoch:0 step:740[D loss: 0.999970] [G loss: 1.000074]
epoch:0 step:745[D loss: 0.999976] [G loss: 1.000073]
epoch:0 step:750[D loss: 0.999967] [G loss: 1.000066]
epoch:0 step:755[D loss: 0.999971] [G loss: 1.000061]
epoch:0 step:760[D loss: 0.999972] [G loss: 1.000060]
epoch:0 step:765[D loss: 0.999974] [G loss: 1.000070]
epoch:0 step:770[D loss: 0.999969] [G loss: 1.000065]
epoch:0 step:775[D loss: 0.999973] [G loss: 1.000073]
epoch:0 step:780[D loss: 0.999971] [G loss: 1.000064]
epoch:0 step:785[D loss: 0.999970] [G loss: 1.000075]
epoch:0 step:790[D loss: 0.999972] [G loss: 1.000071]
epoch:0 step:795[D loss: 0.9

...

epoch:19 step:90925[D loss: 1.000195] [G loss: 0.999771]
epoch:19 step:90930[D loss: 1.000122] [G loss: 1.000000]
epoch:19 step:90935[D loss: 1.000033] [G loss: 0.999981]
epoch:19 step:90940[D loss: 1.000238] [G loss: 0.999933]
epoch:19 step:90945[D loss: 1.000177] [G loss: 1.000319]
epoch:19 step:90950[D loss: 1.000083] [G loss: 1.000257]
epoch:19 step:90955[D loss: 1.000140] [G loss: 1.000216]
epoch:19 step:90960[D loss: 0.999828] [G loss: 1.000321]
epoch:19 step:90965[D loss: 0.999709] [G loss: 1.000258]
epoch:19 step:90970[D loss: 0.999899] [G loss: 1.000156]
epoch:19 step:90975[D loss: 0.999908] [G loss: 1.000085]
epoch:19 step:90980[D loss: 1.000000] [G loss: 0.999979]
epoch:19 step:90985[D loss: 1.000044] [G loss: 0.999982]
epoch:19 step:90990[D loss: 1.000066] [G loss: 0.999889]
epoch:19 step:90995[D loss: 1.000144] [G loss: 0.999885]
epoch:19 step:91000[D loss: 1.000099] [G loss: 0.999860]
##############
[2.73460767 0.81650772 6.94214499 5.34630464 3.64440315 5.67474917
 4.860

epoch:19 step:91605[D loss: 0.999910] [G loss: 1.000000]
epoch:19 step:91610[D loss: 1.000002] [G loss: 0.999918]
epoch:19 step:91615[D loss: 0.999966] [G loss: 1.000121]
epoch:19 step:91620[D loss: 0.999975] [G loss: 1.000084]
epoch:19 step:91625[D loss: 0.999962] [G loss: 1.000219]
epoch:19 step:91630[D loss: 0.999934] [G loss: 1.000262]
epoch:19 step:91635[D loss: 0.999996] [G loss: 1.000243]
epoch:19 step:91640[D loss: 1.000053] [G loss: 1.000324]
epoch:19 step:91645[D loss: 1.000078] [G loss: 1.000389]
epoch:19 step:91650[D loss: 0.999912] [G loss: 1.000172]
epoch:19 step:91655[D loss: 1.000159] [G loss: 0.999706]
epoch:19 step:91660[D loss: 1.000322] [G loss: 0.999510]
epoch:19 step:91665[D loss: 1.000342] [G loss: 0.999381]
epoch:19 step:91670[D loss: 1.000353] [G loss: 0.999335]
epoch:19 step:91675[D loss: 1.000057] [G loss: 0.999638]
epoch:19 step:91680[D loss: 1.000206] [G loss: 0.999761]
epoch:19 step:91685[D loss: 1.000360] [G loss: 0.999847]
epoch:19 step:91690[D loss: 0.9

epoch:19 step:92295[D loss: 0.999960] [G loss: 0.999964]
epoch:19 step:92300[D loss: 0.999883] [G loss: 1.000015]
epoch:19 step:92305[D loss: 0.999894] [G loss: 1.000065]
epoch:19 step:92310[D loss: 0.999956] [G loss: 1.000190]
epoch:19 step:92315[D loss: 0.999926] [G loss: 1.000195]
epoch:19 step:92320[D loss: 0.999989] [G loss: 1.000157]
epoch:19 step:92325[D loss: 0.999937] [G loss: 1.000165]
epoch:19 step:92330[D loss: 1.000024] [G loss: 1.000156]
epoch:19 step:92335[D loss: 1.000180] [G loss: 1.000110]
epoch:19 step:92340[D loss: 1.000155] [G loss: 1.000151]
epoch:19 step:92345[D loss: 1.000131] [G loss: 0.999803]
epoch:19 step:92350[D loss: 1.000299] [G loss: 0.999641]
epoch:19 step:92355[D loss: 1.000301] [G loss: 0.999568]
epoch:19 step:92360[D loss: 1.000339] [G loss: 0.999748]
epoch:19 step:92365[D loss: 1.000316] [G loss: 0.999967]
epoch:19 step:92370[D loss: 1.000215] [G loss: 1.000098]
epoch:19 step:92375[D loss: 0.999731] [G loss: 1.000485]
epoch:19 step:92380[D loss: 0.9

epoch:19 step:92985[D loss: 1.000110] [G loss: 1.000134]
epoch:19 step:92990[D loss: 1.000098] [G loss: 0.999954]
epoch:19 step:92995[D loss: 1.000122] [G loss: 1.000271]
epoch:19 step:93000[D loss: 0.999952] [G loss: 1.000303]
##############
[2.66002795 0.93940635 6.91810224 5.37178544 3.75791434 5.88422381
 5.09566149 4.97463747 5.48664946 3.69498878]
##########
epoch:19 step:93005[D loss: 1.000038] [G loss: 1.000259]
epoch:19 step:93010[D loss: 0.999937] [G loss: 1.000411]
epoch:19 step:93015[D loss: 0.999957] [G loss: 1.000233]
epoch:19 step:93020[D loss: 1.000106] [G loss: 1.000266]
epoch:19 step:93025[D loss: 1.000174] [G loss: 1.000053]
epoch:19 step:93030[D loss: 1.000018] [G loss: 1.000246]
epoch:19 step:93035[D loss: 0.999945] [G loss: 0.999939]
epoch:19 step:93040[D loss: 1.000074] [G loss: 0.999705]
epoch:19 step:93045[D loss: 1.000472] [G loss: 0.999360]
epoch:19 step:93050[D loss: 1.000202] [G loss: 0.999654]
epoch:19 step:93055[D loss: 1.000283] [G loss: 0.999592]
epoch:

epoch:19 step:93655[D loss: 0.999871] [G loss: 1.000619]
epoch:19 step:93660[D loss: 1.000224] [G loss: 1.000679]
epoch:19 step:93665[D loss: 0.999876] [G loss: 1.000532]
epoch:19 step:93670[D loss: 0.999822] [G loss: 1.000294]
epoch:19 step:93675[D loss: 0.999925] [G loss: 1.000165]
epoch:19 step:93680[D loss: 0.999997] [G loss: 1.000101]
epoch:19 step:93685[D loss: 1.000103] [G loss: 1.000054]
epoch:19 step:93690[D loss: 1.000158] [G loss: 0.999975]
epoch:19 step:93695[D loss: 1.000103] [G loss: 1.000094]
epoch:19 step:93700[D loss: 1.000639] [G loss: 0.999840]
